# **<mark>Filter<mark>**
## "Filter"腳本能對資料庫DataBase指定欄位的值進行篩選，以精簡資料內容
因資料量龐大，儲存空間有限，處理資料也相當耗時，在下載資料前請務必進行資料篩選

## 使用說明:
1.請將所有<font color=blue>要篩選的資料表DataBase</font>放到<font color=#0099ff>input資料夾</font>內，Filter會將input資料夾內的<font color=#0099ff>所有資料表</font>合併並進行篩選  
2.若使用病歷號ID或影像號RIS進行篩選，請上傳在DataDownload本身資料夾，<font color=red>不要</font>放到<font color=#0099ff>input資料夾</font>內  
3.篩選結果會存放在<font color=green>output資料夾</font>內，包含<font color=green>篩選後的資料表DataBase</font>以及<font color=green>篩選後的影像清單sorting_list</font>  
4.請<font color=violet>注意大小寫</font>  
5.關鍵字前後請以<font color=violet>符號 ' or "</font> 包括  ex.'text' or "text"  
6.若不確定篩選條件，請先下載<font color=green>影像清單sorting_list</font> ，查看各欄位內容再設定篩選條件
    
    
註1:date的篩選欄位是StudyDate  
註2:station篩選的視StationName，各scanner名稱代號對照 ERCT=急診的CT；CT2F=2樓檢查室的CT；VCT=情人湖CT；MR3T=Siemens的3T MR；MR15=Philips的1.5T MR  
註3:sequence的篩選欄位是SeriesDescription  
註4:description的篩選欄位是PerformedProcedureStepDescription

## <font color=red>注意事項:
<font color=red>★ 執行篩選前請先以"DataBase"腳本建立資料庫  
★ 腳本請依序使用以減少錯誤: 以"DataBase"腳本建立對應的影像資料庫>>以"Filter"腳本進行篩選(以免資料量過於龐大)>>以"Download"腳本能進行下載  
★ 腳本執行後會紀錄上次的執行內容，若發生錯誤，請找到原始的檔案DataDownload.zip，解壓縮後置換換掉整個DataDownload資料夾(詳細方法請參考操作說明-Servers)  
★ 置換DataDownload資料夾前請備份您的重要資料表  
★ 腳本產生的檔案中，<font color=violet>資料庫檔案DataBase_xxx</font>是給腳本使用的資料表，而<font color=violet>影像清單sorting_list</font>則是方便給使用者查看影像內容的資料表，
    使用腳本時請務必放入正確的檔案  </font>

# **<font color=red>腳本使用方法:<p><font color=red>滑鼠選擇要執行的對話框，使對話框左邊顯示<font color=blue>藍色粗邊框<font color=red>後，按下ctrl+enter執行該對話框</font>**

# Step 1: 匯入功能包
(匯入成功會有提示)

In [1]:
#Step 1: import packages  (選取本對話框並按下ctrl+enter執行)
from utils.Functions import *
from utils.get import get_filted_col_by_keys,get_filted_database,get_save_file_name  #2022/7/13 update
from utils import *

DEFAULT PARAMETERS IMPROT SCCESS!
對話框回報內容: 紅色標示注意事項；亮紫色提示類別；藍色標示input資料夾及input detail資料內容；綠色標示output資料；粉紅色背景表示程式碼錯誤
20220715 17:33:14 PACKAGES IMPORT SUCCESS!
請執行下一個步驟


# Step 2: 請先將要篩選的資料表DataBase放到input資料夾中，
# 接著在以下對話框中設定要篩選的條件，若該欄位不篩選，請設為預設值ALL
(可直接將資料庫csv檔拖曳至左側的input資料夾)  
(雙擊對話框可進行編輯)  
(執行Step 2對話框後，請確認參數內容，無誤則執行Step 3，參數錯誤請修改並重新執行Step 2)

In [7]:
#### Step 2: 請設定篩選條件，若該選項無指定條件請指定為預設值ALL，設定完請按下ctrl+enter執行本對話框
sex=ALL  #可指定性別  optins={ALL,F,M}
age=ALL  #optins={ALL,age,(min,max)}可輸入最小與最大值(必須為整數)  ex.20歲: 20；60-70歲: (60,70)
date=ALL  #單日或日期區間*註1  optins={ALL,yyyymmdd,(min,max)}日期格式為8位數字  ex.20220102單日；(20220101,20220112)表示從2022/1/1到2022/1/12
modality=ALL  #可指定檢查類型  options={ALL,CT,MR}
station=ALL  #可選擇scanner*註2  options={ALL,ERCT,CT2F,VCT,MR3T,MR15}
sequence=ALL  #可篩選包含指定關鍵字的序列*註3  ex. 'T1','T2','flair','FLAIR'....
description=ALL    #可篩選包含指定關鍵字的影像*註4  ex.'CTA of brain','without Co','w/o Contr','MRA'

ID=ALL  #若需指定病歷號請改為CSV   optins={ALL,CSV}
RIS=ALL   #若需指定AccessionNumber影像號請改為CSV   optins={ALL,CSV}
my_CSV='my_CSV_template_220715.csv'  #若有指定病歷號或RIS，請上傳資料表並指定資料表路徑及欄位名稱
column_PatientID='ID'  #若有指定病歷號，請輸入資料表內的病歷號欄位名稱
column_RIS='RIS'  #若有指定RIS，請輸入資料表內的RIS欄位名稱


#底下幾行不用改
filter_keys=[ID,sex,age,date,modality,station,sequence,description,RIS]  #2022/7/15: default新增RIS篩選；將patientID篩選從optional移至開放設定區
if ID.upper()==CSV:
    filter_keys[0]=column_PatientID
if RIS.upper()==CSV:
    filter_keys[-1]=column_RIS
print(check_DB.format(BlueLight.format(input_path),'\n{}\n={}'.format(filter_cols,PurpleLight.format(filter_keys)),PurpleLight.format('\nfilted')),Red.format(check_above),sep='\n')

資料路徑: input
即將建立 
['PatientID', 'PatientSex', 'PatientAge', 'StudyDate', 'Modality', 'StationName', 'SeriesDescription', 'PerformedProcedureStepDescription', 'AccessionNumber']
=['ID', 'ALL', 'ALL', 'ALL', 'ALL', 'ALL', 'ALL', 'ALL', 'ALL'] 
filted 的資料庫
請確認上述內容無誤後再執行下一步驟


# Step 3: 篩選資料庫，並產生filted_DataBase以及filted_sorting_list
(執行Step 3對話框後會出現進度條，請務必等待進度完成，結束後會有 <font color=red>執行結束!</font> 的提示)

In [9]:
#Step 3: filter input DataBase by selected conditions, and create filted DataBase and filted sorting_list  (選取本對話框並按下ctrl+enter執行)
#用CSV篩選patient病歷號 & RIS影像號的功能之相關判讀  (2022/7/15新增)
if ID.upper()==CSV:
    input_CSV=pd.read_csv(my_CSV,dtype='str').fillna('')
    patient=list(input_CSV[column_PatientID])
if RIS.upper()==CSV:
    input_CSV=pd.read_csv(my_CSV,dtype='str').fillna('')
    accession=list(input_CSV[column_RIS])

#optional/Advanced settings:
# input_path='input'  #[optional]輸入路徑，篩選前資料庫DataBase的資料表的存放位置
# patient='175623'  #[Advanced]=ID可指定病歷號，可以單一病歷號或以list的形式輸入  ex. 12345 or '12345' or [14267,'51238']  #2022/7/15: 將patientID篩選從optional移至開放設定區
# accession='19CC20531X01'  #[Advanced]可指定AccessionNumber影像號，可以單一影像號str或以list of str的形式輸入  ex. '19CC20531X01' or ['19CC20531X01','RIS2']  #2022/7/15新增RIS篩選
Same=False  #[optional]if need, change Same to 'True' to get accurate samples with assigned keys(Same是針對文字類型的欄位篩選，設定False為"包含"關鍵字，設定True則是與關鍵字"完全相同"的資料)

#define output file names:  #2022/7/13 update
save_filted_csv_name,save_filted_sorting_csv_name=get_save_file_name(output_path,Prefix='filted_')
print('saving:',save_filted_csv_name,save_filted_sorting_csv_name,waiting)
   
#建立篩選條件列表: 
# filter_cols=['PatientID','PatientSex','PatientAge','StudyDate','Modality','StationName','SeriesDescription','PerformedProcedureStepDescription','AccessionNumber']  #若變更DB欄位或新增篩選條件須重新定義filter_cols
filter_keys=[patient,sex,age,date,modality,station,sequence,description,accession]  #若調整參數內容須重新抓filter_keys
filted_col,filted_key=get_filted_col_by_keys(filter_cols,filter_keys)

#讀取篩選前資料庫並篩選資料庫filted_DataBase:
DB_path_list=GetInputDir(input_path,filetype='.csv')
for d in DB_path_list:   #jupyter會自動建立checkpoints導致混亂QQ
    if 'ipynb_checkpoints' in d:
        DB_path_list.remove(d)
print(Blue.format('input資料表={}'.format(DB_path_list)))
DB=GetConcatedDF(DB_path_list,cols=False,replace_value='NotInDict',ignore_missing=True,replacer='')
print(Blue.format('資料表大小: {}'.format(DB.shape)))
# DB=test_DB  #test
filted_DB=get_filted_database(Input_DB=DB,Filted_key=filted_key,Filted_col=filted_col,Save_csv_name=save_filted_csv_name,Same=Same)  #篩選DB
print(built.format('filted DataBase'))

#建立filted_sorting_list:
Input_DB=filted_DB
Output_sorting_list=GetUnique(Input_DB,df_sorting)
Output_sorting_list.to_csv(save_filted_sorting_csv_name,index=0)
print(save.format(sort_build+': {} {}'.format(Output_sorting_list.shape,GreenItalic.format(save_filted_sorting_csv_name))),Red.format(done))

saving: output/filted_DataBase-20220715173549.csv output/filted_sorting_list-20220715173549.csv 請稍等...
input資料表=['input/DataBase_anonymous_202107-20220713162204.csv', 'input/DataBase_test.csv']
資料表大小: (162, 38)
(1, 38) output/filted_DataBase-20220715173549.csv saved
filted DataBase已建立
sorting_list已建立: (1, 13) output/filted_sorting_list-20220715173549.csv saved 
執行結束!


# Step 4: 請至<font color=green>output資料夾</font>查看結果

## \~底下沒東西了~

<font size="4">版本資訊: <font size="3">ver.220715</font><br><font size="2">
    2022/7/15:  
    1.調整說明文字及順序  
    2.新增AccessionNumber篩選功能  
    3.新增以CSV內的ID和RIS篩選patientID及AccessionNumber的功能
    <br>
</font> 